# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
import os 
from dotenv import load_dotenv

price_data_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), '05_src', 'data', 'prices')
print(f"PRICE_DATA environment variable: {price_data_path}")


PRICE_DATA environment variable: /Users/sehroz/Desktop/production/05_src/data/prices


In [2]:
import dask.dataframe as dd

/var/folders/5y/vm0pttr52w79bbdk58s7d92r0000gn/T/ipykernel_12983/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

parquet_files = glob(os.path.join(price_data_path, '**', '*.parquet', 'part.0.parquet'), recursive=True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [9]:
# Write your code below.

dd_feat = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (dd_feat
          .groupby('ticker', group_keys=False)
          .apply(lambda x: x.assign(
              lag_close=x['Close'].shift(1),
              lag_Adj_Close=x['Adj Close'].shift(1)
          ))
          .assign(
              returns=lambda x: x['Adj Close']/x['lag_Adj_Close'] - 1,
              hi_lo_range=lambda x: x['High'] - x['Low']
          ))



/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/var/folders/5y/vm0pttr52w79bbdk58s7d92r0000gn/T/ipykernel_12983/3290736219.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_feat


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [13]:

df_feat = dd_feat.compute()


df_feat['rolling_avg_return'] = df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(window=10).mean())
print(f"\nFinal dataset shape: {df_feat.shape}")

print(df_feat.loc["AAPL"][['returns', 'hi_lo_range', 'rolling_avg_return']].head(12))


Final dataset shape: (2778199, 15)
Price    returns  hi_lo_range  rolling_avg_return
ticker                                           
AAPL         NaN     0.012276                 NaN
AAPL    0.100841     0.040178                 NaN
AAPL    0.041986     0.030134                 NaN
AAPL   -0.040294     0.023438                 NaN
AAPL    0.011450     0.018694                 NaN
AAPL    0.037737     0.019252                 NaN
AAPL   -0.036364     0.016741                 NaN
AAPL    0.086794     0.040178                 NaN
AAPL   -0.045140     0.016741                 NaN
AAPL   -0.003636     0.022322                 NaN
AAPL   -0.018251     0.018973            0.013512
AAPL    0.111524     0.020089            0.014581


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [15]:
## 1) Not necessary to convert to pandas. Dask can do rolling averages too.

## 2) Only if we were dealing with more data or needed parallel processing.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [x] Created a branch with the correct naming convention.
- [x] Ensured that the repository is public.
- [x] Reviewed the PR description guidelines and adhered to them.
- [x] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.